In [1]:
import os
import collections

from gensim.models import KeyedVectors
from keras.callbacks import TensorBoard
from keras.layers import Dense, Dropout, Conv1D, Embedding, GlobalMaxPooling1D
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import nltk
import numpy as np
import codecs

np.random.seed(42)

INPUT_FILE = os.path.join(os.path.dirname("__file__"),
                          "data/training.txt")
GLOVE_MODEL = os.path.join(os.path.dirname("__file__"),
                           "data/glove.6b.300d.txt")
LOG_DIR = os.path.join(os.path.dirname("__file__"), "logs")
VOCAB_SIZE = 5000
EMBED_SIZE = 300
NUM_FILTERS = 256
NUM_WORDS = 3
BATCH_SIZE = 64
# 事前学習済みモデルを使用するため早く収束する
NUM_EPOCHS = 10

Using TensorFlow backend.


In [2]:
counter = collections.Counter()
# file open
with codecs.open(INPUT_FILE, "r", encoding="utf-8") as fin:
    maxlen = 0
    for line in fin:
        # 行をtabでsplit
        _, sent = line.strip().split("\t")
        try:
            # NLTK(Natural Language Toolkit)で単語分割
            words = [x.lower() for x in nltk.word_tokenize(sent)]
        except LookupError:
            print("English tokenize does not downloaded. So download it.")
            nltk.download("punkt")
            words = [x.lower() for x in nltk.word_tokenize(sent)]
        maxlen = max(maxlen, len(words))
        for word in words:
            # 単語ごとでカウント
            counter[word] += 1
            # int型でdictionaryを初期化
            word2index = collections.defaultdict(int)
            # 出現回数が多い順に（文字, 回数）のタプル5000個に対して単語IDを定義
            for wid, word in enumerate(counter.most_common(VOCAB_SIZE)):
                word2index[word[0]] = wid + 1
            vocab_sz = len(word2index) + 1
            index2word = {v: k for k, v in word2index.items()}

In [3]:
xs, ys = [], []
with codecs.open(INPUT_FILE, "r", encoding="utf-8") as fin:
    for line in fin:
        label, sent = line.strip().split("\t")
        ys.append(int(label))
        words = [x.lower() for x in nltk.word_tokenize(sent)]
        wids = [word2index[word] for word in words]
        xs.append(wids)

X = pad_sequences(xs, maxlen = maxlen)
Y = np_utils.to_categorical(ys)

Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size = 0.3, random_state = 42)

In [4]:
word2emb = {}
# GloVeモデルから辞書を作成する
with codecs.open(GLOVE_MODEL, "r", encoding="utf-8") as fglove:
    for line in fglove:
        cols = line.strip().split()
        word = cols[0]
        embedding = np.array(cols[1:], dtype = "float32")
        word2emb[word] = embedding

In [5]:
embedding_weights = np.zeros((vocab_sz, EMBED_SIZE))
for word, index in word2index.items():
    try:
        embedding_weights[index, :] = word2emb[word]
    except KeyError:
        pass

In [6]:
# ネットワークの構築
model = Sequential()
model.add(Embedding(vocab_sz, EMBED_SIZE, input_length = maxlen,
                    weights = [embedding_weights],
                    trainable = True))
model.add(Dropout(0.2))
model.add(Conv1D(filters = NUM_FILTERS, kernel_size = NUM_WORDS,
                 activation = "relu"))
model.add(GlobalMaxPooling1D())
model.add(Dense(2, activation="softmax"))

# コンパイル
model.compile(optimizer = "adam", loss="categorical_crossentropy", metrics = ["accuracy"])

history = model.fit(Xtrain, Ytrain, batch_size = BATCH_SIZE,
                    epochs = NUM_EPOCHS,
                    #callbacks=[TensorBoard(LOG_DIR)],
                    validation_data = (Xtest, Ytest)
                   )

score = model.evaluate(Xtest, Ytest, verbose = 1)
print("Test score: {:.3f}, accuracy: {:.3f}".format(score[0], score[1]))

C:\Users\tetsu\Anaconda3\envs\py_gpu\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 4960 samples, validate on 2126 samples
Epoch 1/10
4960/4960 [==============================] - 4s 740us/step - loss: 0.1280 - accuracy: 0.9454 - val_loss: 0.0331 - val_accuracy: 0.9878
Epoch 2/10
4960/4960 [==============================] - 3s 692us/step - loss: 0.0173 - accuracy: 0.9962 - val_loss: 0.0244 - val_accuracy: 0.9929
Epoch 3/10
4960/4960 [==============================] - 3s 702us/step - loss: 0.0076 - accuracy: 0.9986 - val_loss: 0.0215 - val_accuracy: 0.9920
Epoch 4/10
4960/4960 [==============================] - 4s 715us/step - loss: 0.0047 - accuracy: 0.9992 - val_loss: 0.0190 - val_accuracy: 0.9934
Epoch 5/10
4960/4960 [==============================] - 3s 702us/step - loss: 0.0031 - accuracy: 0.9992 - val_loss: 0.0189 - val_accuracy: 0.9934
Epoch 6/10
4960/4960 [==============================] - 3s 697us/step - loss: 0.0020 - accuracy: 0.9998 - val_loss: 0.0205 - val_accuracy: 0.9929
Epoch 7/10
4960/4960 [==============================] - 3s 697us/step - loss